In [19]:
from IPython.display import Markdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import matthews_corrcoef, average_precision_score, f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

import os
original_dir = os.getcwd()
if os.path.basename(original_dir) == "SupervisedModels":
    os.chdir(os.path.dirname(original_dir))


In [20]:
if 'ShowOutput' in locals() or 'ShowOutput' in globals():
    ShowOutput = ShowOutput
else:
    ShowOutput = False
# Load and clean data
%run dataInfo.ipynb

score_cols = [f"A{i}_Score" for i in range(1, 11)]
train_df['total_score'] = train_df[score_cols].sum(axis=1)
test_df['total_score'] = test_df[score_cols].sum(axis=1)

# Normalize the total score
train_df['score_ratio'] = train_df['total_score'] / 10
test_df['score_ratio'] = test_df['total_score'] / 10

# Add interaction features
train_df['gender_result'] = train_df['gender'] * train_df['result']
train_df['age_score_ratio'] = train_df['age'] * train_df['score_ratio']
train_df['score_autism'] = train_df['total_score'] * train_df['autism']
train_df['age_jaundice'] = train_df['age'] * train_df['jaundice']
train_df['autism_result'] = train_df['autism'] * train_df['result']
train_df['gender_total_score'] = train_df['gender'] * train_df['total_score']

test_df['gender_result'] = test_df['gender'] * test_df['result']
test_df['age_score_ratio'] = test_df['age'] * test_df['score_ratio']
test_df['score_autism'] = test_df['total_score'] * test_df['autism']
test_df['age_jaundice'] = test_df['age'] * test_df['jaundice']
test_df['autism_result'] = test_df['autism'] * test_df['result']
test_df['gender_total_score'] = test_df['gender'] * test_df['total_score']

cleanTrain, cleanTest = train_df, test_df
X = cleanTrain.drop(columns=["Class/ASD"])
y = cleanTrain["Class/ASD"]

X_train_main, X_holdout, y_train_main, y_holdout = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [21]:

def tune_logistic_regression_with_cv(X, y, n_trials=30):
    smote = SMOTE(random_state=42)
    imputer = SimpleImputer(strategy="mean")
    scaler = StandardScaler()
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    def objective(trial):
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
        solver = 'liblinear' if penalty == 'l1' else 'lbfgs'
        params = {
            'C': trial.suggest_float('C', 1e-3, 1e2, log=True),
            'penalty': penalty,
            'solver': solver,  # Supports both l1 and l2
            'max_iter': 1000,
            'random_state': 42
        }

        f1_scores = []
        for train_idx, val_idx in skf.split(X, y):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            X_train = imputer.fit_transform(X_train)
            X_val = imputer.transform(X_val)
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

            X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

            model = LogisticRegression(**params)
            model.fit(X_resampled, y_resampled)
            y_pred = model.predict(X_val)
            f1_scores.append(f1_score(y_val, y_pred))

        return np.mean(f1_scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    if(ShowOutput):
        print("Besthyperparameters:", study.best_params)
    return study.best_params


In [22]:

best_params = tune_logistic_regression_with_cv(X_train_main, y_train_main, n_trials=40)

imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()
X_imputed = imputer.fit_transform(X_train_main)
X_scaled = scaler.fit_transform(X_imputed)
X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_scaled, y_train_main)

lr_model = LogisticRegression(**best_params)
lr_model.fit(X_resampled, y_resampled)


LogisticRegression(C=0.004898378012468877)

In [23]:
X_test = cleanTest.copy()

missing_cols = set(X.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0  

X_test = X_test[X.columns]

imputer.fit(X_test)
scaler.fit(X_test)

X_test = imputer.transform(X_test)
X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns=X.columns)

test_predictions = lr_model.predict(X_test)
test_probabilities = lr_model.predict_proba(X_test)[:, 1]

cleanTest_with_predictions = cleanTest.copy()
cleanTest_with_predictions['Class/ASD'] = test_predictions
cleanTest_with_predictions['Probability'] = test_probabilities

In [ ]:

y_proba = lr_model.predict_proba(X_scaled)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)

final_roc_auc = roc_auc_score(y_train_main, y_pred)
final_pr_auc = average_precision_score(y_train_main, y_pred)
final_f1 = f1_score(y_train_main, y_pred)
final_mcc = matthews_corrcoef(y_train_main, y_pred)
accuracy = accuracy_score(y_train_main, y_pred)

if(ShowOutput):
    # print("F1 Score:", f1_score(y_train_main, y_pred))
    # print("ROC AUC:", roc_auc_score(y_train_main, y_proba))
    # print("Accuracy:", accuracy_score(y_train_main, y_pred))
    # print("Precision:", precision_score(y_train_main, y_pred))
    # print("Recall:", recall_score(y_train_main, y_pred))
    

    
    ConfusionMatrixDisplay.from_predictions(y_train_main, y_pred)
    plt.title("Confusion Matrix - Final Logistic Regression Model")
    plt.show()

    df_metrics = pd.DataFrame({
        "Metric": ["ROC AUC", "PR AUC", "F1 Score", "Matthews Corrcoef", "Accuracy"],
        "Value": [final_roc_auc, final_pr_auc, final_f1, final_mcc, accuracy]
    })

    df_metrics["Value"] = df_metrics["Value"].map("{:.4f}".format)

    display(df_metrics)

,Metric,Value
0,ROC AUC,0.8713
1,PR AUC,0.5525
2,F1 Score,0.7138
3,Matthews Corrcoef,0.6464
4,Accuracy,0.8547
